In [1]:
import json
mwoz_file = "dialogues_016.json"
outfile = "bert_dialogue_016.json"
service_domain = "restaurant"

In [2]:
with open(mwoz_file) as fp:
    mwoz_dialogues_list = json.load(fp)

In [3]:
for mwoz_dialogue in mwoz_dialogues_list:
    print(len(mwoz_dialogues_list))
    print(mwoz_dialogue["dialogue_id"])
    print(len(mwoz_dialogue["turns"]))
    print(len(mwoz_dialogue["turns"][0]))
    print(mwoz_dialogue["turns"][0]["utterance"])
    break

512
MUL1966.json
16
4
Hi, I'm looking for places to visit. Mainly in the western part.


In [4]:
total_dialogs = 0
restaurant_dialogs = 0
json_list = []
for mwoz_dialogue in mwoz_dialogues_list:
    total_dialogs += 1
    if not service_domain in mwoz_dialogue["services"]:
        continue
    
    restaurant_dialogs += 1
    bert_dict = {}
    bert_dict["dialogue_idx"] = mwoz_dialogue["dialogue_id"]
    #bert_dict["domains"] = mwoz_dialogue["services"]
    bert_dict["domains"] = [service_domain]
    bert_dict["dialogue"] = []
    
    # special handling for the 0th element, always being USER
    system_transcript = ""
    turn_idx = 0
    
    belief_state_list = []
    turn_label_list = []
    for dom in mwoz_dialogue["turns"][0]["frames"]:
        act = dom["state"]["active_intent"]
        service = dom["service"]
        # consider only service = "restaurant" (i.e service_domain) and 
        # active_intent not being NONE
        if (not act == "NONE") and (service == service_domain):
            req_slot = dom["state"]["requested_slots"]
            slot_val_mwoz = dom["state"]["slot_values"]
            for key, val in slot_val_mwoz.items():
                belief_state_list.append({"slots": [[key, val[0]]], "act": act})
                turn_label_list.append([key, val[0]])
            
    transcript = mwoz_dialogue["turns"][0]["utterance"]
    system_acts = req_slot
    domain = service_domain
    
    bert_dialog_dict = {"system_transcript": system_transcript, "turn_idx": turn_idx,
                        "belief_state": belief_state_list, "turn_label": turn_label_list, 
                        "transcript": transcript, "system_acts": system_acts, "domain": domain
                       }
    bert_dict["dialogue"].append(bert_dialog_dict)
    
    # for entries in "turn" from 1 to n-1, where n is the final utterance
    # consider index 1 ("SYSTEM"), 2 ("USER") paired; take slots and value
    # from utterance 2 (USER).
    i = 1
    turn_len = len(mwoz_dialogue["turns"])
    while i <= turn_len - 2:
        utter_sys = mwoz_dialogue["turns"][i] # represents SYSTEM
        utter_user = mwoz_dialogue["turns"][i+1] # represents USER
        
        system_transcript = utter_sys["utterance"] # system utterance
        turn_idx += 1
        
        belief_state_list = []
        turn_label_list = []
        for dom in utter_user["frames"]:
            act = dom["state"]["active_intent"]
            service = dom["service"]
            # consider only service = "restaurant" (i.e service_domain) and 
            # active_intent not being NONE
            if (not act == "NONE") and (service == service_domain):
                slot_val_mwoz = dom["state"]["slot_values"]
                req_slot = dom["state"]["requested_slots"]
                for key, val in slot_val_mwoz.items():
                    belief_state_list.append({"slots": [[key, val[0]]], "act": act})
                    turn_label_list.append([key, val[0]])
        
        transcript = utter_user["utterance"] 
        system_acts = req_slot
        domain = service_domain
        
        bert_dialog_dict = {"system_transcript": system_transcript, "turn_idx": turn_idx,
                            "belief_state": belief_state_list, "turn_label": turn_label_list, 
                            "transcript": transcript, "system_acts": system_acts, "domain": domain
                           }
        
        bert_dict["dialogue"].append(bert_dialog_dict)
        
        i += 2
        
    # handle the case of the last system utterance
    utter_sys = mwoz_dialogue["turns"][i]
    system_transcript = utter_sys["utterance"] # system utterance
    turn_idx += 1
        
    belief_state_list = []
    turn_label_list = []
        
    transcript = ""
    system_acts = []
    domain = service_domain
        
    bert_dialog_dict = {"system_transcript": system_transcript, "turn_idx": turn_idx,
                        "belief_state": belief_state_list, "turn_label": turn_label_list, 
                        "transcript": transcript, "system_acts": system_acts, "domain": domain
                       }
    bert_dict["dialogue"].append(bert_dialog_dict)
        
    json_list.append(bert_dict)

print("Total dialogs:", total_dialogs, "Restaurant dialogs:", restaurant_dialogs)

Total dialogs: 512 Restaurant dialogs: 234


In [5]:
json_object = json.dumps(json_list, indent=2)

with open(outfile, "w") as fp:
    fp.write(json_object)